# Portfoliobenadering betrouwbaarheidseisen
Faalfrequenties en betrouwbaarheidseisen voor grote aantallen vergelijkbare constructies in relatie tot de NEN 8700.

## Uitgangspunten en definities

### Grenstoestandsfunctie
$ Z = p * m_R * R - a_Q * m_Q * Q - (1 - a_Q) * G$

met 
- $p$ = design parameter (verhouding sterkte/belasting)
- $m_R$ = modelonzekerheid sterkte (tijdsonafhankelijk, volledig gecorreleerd in de tijd; volledige correlatie tussen constructies)
- $R$ = sterkte (tijdsonafhankelijk, volledig gecorreleerd in de tijd; geen correlatie tussen constructies)
- $a_Q$ = aandeel veranderlijke belasting ($=mu_Q/mu_G$)
- $m_Q$ = modelonzekerheid veranderlijke belasting (tijdsonafhankelijk, volledig gecorreleerd in de tijd; volledige correlatie tussen constructies)
- $Q$ = veranderlijke belasting (tijdsafhankelijk, ongecorreleerd tussen jaren; geen correlatie tussen constructies)
- $G$ = permanente belasting (eigen gewicht; volledig gecorreleerd in de tijd; geen correlatie tussen constructies)

In [3]:
def LimitStateFunction(p, mR, R, aQ, mQ, Q, G):
    Z = p * mR * R - aQ *  mQ *Q - (1-aQ) * G
    return Z

### Unity check
$ UC = p * R_{kar} / \gamma_R - a_Q * Q_{kar} * \gamma_Q - (1 - a_Q) * G_{kar}$

met 
- $p$ = design parameter (verhouding sterkte/belasting)
- $R_{kar}$ = karakteristieke waarde sterkte (5%-kwantiel $R$)
- $a_Q$ = aandeel veranderlijke belasting
- $Q_{kar}$ = karakteristieke waarde veranderlijke belasting (98%-kwantiel $Q$)
- $G_{kar}$ = karakteristieke waarde permanente belasting (50%-kwantiel $G$)

In [4]:
from scipy.stats import norm

def UnityCheck(p, V_R, gamma_R, V_Q, gamma_Q):
    R_kar = 1 - 1.645 * V_R
    Q_kar = norm.ppf(0.98, loc = 1, scale = V_Q)
    G_kar = 1
    UC = (aQ * Q_kar * gamma_Q + (1-aQ) * G_kar) / (p * R_kar / gamma_R)
    return UC

**Determine p for given UC**

In [5]:
def Get_P_for_UC(UC, V_R, gamma_R, V_Q, gamma_Q):
    R_kar = 1 - 1.645 * V_R
    Q_kar = norm.ppf(0.98, loc = 1, scale = V_Q)
    G_kar = 1
    p =  (aQ * Q_kar * gamma_Q + (1-aQ) * G_kar) / (UC * R_kar / gamma_R)
    return p

### Variatiecoefficienten
voor genormaliseerde stochasten (c.q. gemiddelden gelijk aan 1)

In [6]:
V_mR = 0.12
V_R = 0.15
V_mQ = 0.1
V_Q = 0.15
V_G = 0.05

### Partiele factoren

In [7]:
gamma_R = 1.1
gamma_Q = 1.05

## Analyse
### Bereken unity check (UC)

In [8]:
aQ = 0.1
p = 1.50
UC = UnityCheck(p, V_R, gamma_R, V_Q, gamma_Q)
print('UC = ', UC)

UC =  1.009918088174972


### Faalkansanalyse met Probabilistic Toolkit (PTK)

In [9]:
import sys, os
sys.path.append("C:\Program Files (x86)\Deltares\Probabilistic toolkit\Python")
import toolkit

PTKfile = 'Portfolio_v1'

toolkit.Initialize()
toolkit.Load(os.path.abspath(f"{PTKfile}.tkx"))
toolkit.SetVariableValue('p', "Mean", p)
toolkit.SetVariableValue('aQ', "Mean", aQ)
toolkit.Run()

# retrieve results
vars = toolkit.GetVariables()

beta = toolkit.GetReliabilityIndex()
Pf = toolkit.GetProbabilityFailure()
alphas=[]
DPvalues=[]
for var in vars[:-1]:
    alphas.append(toolkit.GetAlpha(var))
    DPvalues.append(toolkit.GetAlphaPhysical(var))

results = {"p": [],
            "aQ": [],
            "UC": [],
            "Beta": [],
            "Pf": [],
            "alphas": [],
            "DPvalues": [],
            "VarNames": []}

results["p"].append(p)
results["aQ"].append(aQ)
results["UC"].append(float(UC))
results["Beta"].append(float(beta))
results["Pf"].append(float(Pf))
results["alphas"].append(list(map(float, alphas)))
results["DPvalues"].append(list(map(float, DPvalues)))
results["VarNames"].append(vars)

toolkit.Exit()

print('beta = ', beta)
print('Pf = ', Pf)


beta =  1.74945167152741
Pf =  0.0401064446634775
